Heatmap da linha 409
=================

O heatmap compartilhado anteriormente foi um primeiro passo na visualização dos dados do serviço de gps:

![Linha 409](409_zoom.png)

 Além dos aspectos relativos ao objetivo da verificação do serviço de ônibus, o gráfico apresenta uma visão limitada: não é possível distinguir claramente a densidade dos sinais nas áreas onde os ônibus trafegam com maior frequencia.

In [1]:
import pandas as pd
import datashader as ds

from pyproj import Proj, transform

from bokeh.models import BoxZoomTool
from bokeh.plotting import figure, output_notebook

from datashader.bokeh_ext import InteractiveImage
from datashader.colors import Hot
from datashader import transfer_functions as tf

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
gps_data = pd.read_csv('409_clean.csv')

In [4]:
input_proj = Proj(init='epsg:4326')
output_proj = Proj(init='epsg:3857')

projected_lat = list()
projected_long = list()

for item in zip(gps_data['lat'].values, gps_data['long'].values):
    x, y = transform(input_proj, output_proj, item[1], item[0])
    projected_long.append(x)
    projected_lat.append(y)
    
gps_data['long'] = projected_long
gps_data['lat'] = projected_lat

In [5]:
gps_data['signals'] = 1

In [6]:
x_range, y_range = ((min(gps_data['long']),max(gps_data['long'])), (min(gps_data['lat']),max(gps_data['lat'])))
plot_width  = int(750)
plot_height = int(plot_width//1.2)

In [7]:
def base_plot(tools='pan,wheel_zoom,reset,save',plot_width=plot_width, plot_height=plot_height, **plot_args):
    p = figure(tools=tools, plot_width=plot_width, plot_height=plot_height,
        x_range=x_range, y_range=y_range, outline_line_color=None,
        min_border=0, min_border_left=0, min_border_right=0,
        min_border_top=0, min_border_bottom=0, **plot_args)
    
    p.axis.visible = False
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    
    p.add_tools(BoxZoomTool(match_aspect=True))
    
    return p

In [8]:
def create_image(x_range, y_range, w=plot_width, h=plot_height):
    cvs = ds.Canvas(plot_width=w, plot_height=h, x_range=x_range, y_range=y_range)
    agg = cvs.points(gps_data, 'long', 'lat',  ds.count('signals'))
    img = tf.shade(agg, cmap=Hot, how='eq_hist')
    return tf.dynspread(img, threshold=0.5, max_px=4)

In [9]:
background = "black"
p = base_plot(background_fill_color=background)
InteractiveImage(p, create_image)

![heatmap](bokeh_plot.png)

A visualização acima deixa muito evidente a diferença entre as áreas com mais tráfego, especialmente a área próxima ao ponto final no bairro da Tijuca.